In [1]:
%%configure -f
{"executorCores": 4, "executorMemory": "9486M"}

In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer
import html2text
from smart_open import smart_open
from time import time
from tqdm import tqdm
import concurrent.futures

import json

from itertools import islice

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

import scrapy
from scrapy.linkextractors import LinkExtractor
import html2text
from urllib.parse import urlparse, ParseResult
from scrapy.linkextractors import IGNORED_EXTENSIONS
from scrapy.selector import Selector

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
14,application_1564479621651_0015,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
input_file = "s3://onai-ml-dev-eu-west-1/web_crawler/data/seed_urls/company-urls.csv"

def extract_text_from_html(html, upper_bound=10000):
    if len(html) > upper_bound:
        html =  html[:upper_bound]
    parser = html2text.HTML2Text()
    parser.wrap_links = False
    parser.skip_internal_links = True
    parser.inline_links = True
    parser.ignore_anchors = True
    parser.ignore_images = True
    parser.ignore_emphasis = True
    parser.ignore_links = True
    return parser.handle(html)

def convert_url(url):
    p = urlparse(url, 'http')
    netloc = p.netloc or p.path
    path = p.path if p.netloc else ''
    if not netloc.startswith('www.'):
        netloc = 'www.' + netloc

    p = ParseResult('http', netloc, path, *p[3:])
    return p.geturl()

class MySpider(scrapy.Spider):
    name = "my_spider"

    # crawl in BFS
    custom_settings = {
        'DEPTH_LIMIT': 2,
        'DEPTH_PRIORITY': 1,
        'SCHEDULER_DISK_QUEUE': 'scrapy.squeues.PickleFifoDiskQueue',
        'SCHEDULER_MEMORY_QUEUE': 'scrapy.squeues.FifoMemoryQueue',
        'SCHEDULER_PRIORITY_QUEUE': 'scrapy.pqueues.DownloaderAwarePriorityQueue',
        'DNS_TIMEOUT': 20,
        'DOWNLOAD_TIMEOUT': 30,
        'LOG_LEVEL': 'ERROR',
        'CONCURRENT_REQUESTS': 100,
        'REACTOR_THREADPOOL_MAXSIZE': 20,        
        'REDIRECT_ENABLED': False,
        'RETRY_ENABLED': False,
    }

    def __init__(self, *args, **kwargs):
#         runners.add(1)
        start_urls = kwargs.get('start_urls')
        parsed_urls = [urlparse(start_url) for start_url in start_urls]
        url_domains = [parsed_url.netloc for parsed_url in parsed_urls]
        self.allowed_domains = url_domains

        self.output = kwargs.get('output')

        self.allowed_extensions = ('.pdf', '.doc', '.xls', '.docx', '.xlsx')
        self.denied_extensions = tuple(set(['.' + ext for ext in IGNORED_EXTENSIONS]) - set(self.allowed_extensions))

        super(MySpider, self).__init__(*args, **kwargs)

    def save_file(self, response, upper_limit=2000000): # ~ 2 mbs tops
        page_url = response.meta.get('page_url')
        root_title = response.meta.get("root_title")

        path = response.url.split('/')[-1]
        filename = path[path.rfind("/") + 1:]
#         print("File:", filename)
        if len(response.body) < upper_limit:
            self.output.append((root_title, page_url, filename, "file", bytearray(response.body)))

    def parse(self, response):
        title_xpath = response.selector.xpath('//title/text()').extract()
        
        page_title = title_xpath[0] if title_xpath else ""
        
        root_title = response.meta.get("root_title")
        if root_title is None:
            root_title = page_title
        
        page_content = extract_text_from_html(response.text)
        self.log('Page: %s (%s)' % (response.url, page_title))

        extracted_links = LinkExtractor().extract_links(response)
        for link in extracted_links:
            url = link.url
            parsed_url = urlparse(url)
            url_domain = parsed_url.netloc
            
            follow_link = False
            for marker in ['who we are', 'overview', 'about', 'mission']:
                if marker in link.url.lower():
                    follow_link = True

            for marker in ['who we are', 'overview', 'about', 'mission']:
                if marker in link.text.lower():
                    follow_link = True
                    
            if not follow_link:
                continue

            if url_domain in self.allowed_domains:
                if link.url.endswith(self.allowed_extensions):
                    yield scrapy.Request(url,
                                         callback=self.save_file,
                                         meta={'page_url': response.url, 
                                               'root_title':root_title
                                              }
                                        )
                elif link.url.endswith(self.denied_extensions):
                    continue
                else:
                    yield scrapy.Request(url, callback=self.parse, meta={'root_title': root_title})
        
        self.output.append((root_title, response.url, page_title, "html", page_content))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
pandas_df = pd.read_csv(input_file, header=0, sep='\t').astype(str)
df = spark.createDataFrame(pandas_df).repartition(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def run_scraper(urls):
    from multiprocessing import Process, Queue
    
    import twisted.internet.reactor as reactor
    from twisted.internet import default, defer, task
    from scrapy.crawler import CrawlerRunner

    def f(q, output_q):
        try:
            data = []
            runner = CrawlerRunner()
            deferred = runner.crawl(MySpider, start_urls=[convert_url(url) for url in urls], output=data)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
            output_q.put(data)
            raise ValueError(data)
        except Exception as e:
            q.put(e)

    q = Queue()
    output_q = Queue()
    p = Process(target=f, args=(q, output_q))
    p.start()
    error = q.get()
    output = output_q.get()
    p.join()

    if error is not None:
        raise error

    return output

def run_spider(urls):
    from twisted.internet import reactor
    from scrapy.crawler import CrawlerRunner
    
    from crochet import setup, wait_for, TimeoutError
    setup()
    
    data = []
    
    @wait_for(timeout=200.0)
    def run_it_already():
        crawler = CrawlerRunner()
        return crawler.crawl(MySpider, start_urls=[convert_url(url) for url in urls], output=data)
    
    try:
        run_it_already()
    except TimeoutError:
        # In case of timeout, just return what we managed to scrape
        print("Timed out for given URLs", list(urls))
        print(data)

    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
tst_urls = pandas_df["url"].iloc[:100]

In [7]:
%%time
data1 = run_spider(tst_urls)

INFO:twisted:Log opened.
INFO:scrapy.crawler:Overridden settings: {'CONCURRENT_REQUESTS': 100, 'DEPTH_LIMIT': 2, 'DEPTH_PRIORITY': 1, 'DNS_TIMEOUT': 20, 'DOWNLOAD_TIMEOUT': 30, 'LOG_LEVEL': 'ERROR', 'REACTOR_THREADPOOL_MAXSIZE': 20, 'REDIRECT_ENABLED': False, 'RETRY_ENABLED': False, 'SCHEDULER_DISK_QUEUE': 'scrapy.squeues.PickleFifoDiskQueue', 'SCHEDULER_MEMORY_QUEUE': 'scrapy.squeues.FifoMemoryQueue', 'SCHEDULER_PRIORITY_QUEUE': 'scrapy.pqueues.DownloaderAwarePriorityQueue'}
INFO:scrapy.extensions.telnet:Telnet Password: 22a60bc0b7dc693d
INFO:scrapy.middleware:Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
INFO:scrapy.middleware:Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.Default

DEBUG:scrapy.core.engine:Crawled (301) <GET http://www.amaiowa.com> (referer: None)
DEBUG:scrapy.core.engine:Crawled (200) <GET http://www.sol-distribution.co.uk> (referer: None)
DEBUG:scrapy.core.engine:Crawled (302) <GET http://www.expresspersonel.com> (referer: None)
DEBUG:scrapy.core.engine:Crawled (200) <GET http://www.h2glenfern.com> (referer: None)
DEBUG:scrapy.core.engine:Crawled (301) <GET http://www.patientsafetyauthority.org> (referer: None)
INFO:scrapy.spidermiddlewares.httperror:Ignoring response <301 http://www.flo-2d.com>: HTTP status code is not handled or not allowed
INFO:scrapy.spidermiddlewares.httperror:Ignoring response <302 http://www.thei3p.org>: HTTP status code is not handled or not allowed
INFO:scrapy.spidermiddlewares.httperror:Ignoring response <301 http://www.vizualms.co.uk>: HTTP status code is not handled or not allowed
ERROR:scrapy.core.scraper:Error downloading <GET http://www.appaturemobile.com>
Traceback (most recent call last):
  File "/usr/local/lib

DEBUG:scrapy.core.engine:Crawled (200) <GET http://www.takeitglobal.co.uk> (referer: None)
DEBUG:scrapy.core.engine:Crawled (200) <GET http://www.bestinc.org> (referer: None)
DEBUG:scrapy.core.engine:Crawled (200) <GET http://www.rowekamp.com> (referer: None)
DEBUG:scrapy.core.engine:Crawled (301) <GET http://www.davidmeermanscott.com> (referer: None)
DEBUG:scrapy.core.engine:Crawled (200) <GET http://www.waltonpost.com> (referer: None)
INFO:scrapy.spidermiddlewares.httperror:Ignoring response <301 http://www.wellnessworkdays.com>: HTTP status code is not handled or not allowed
INFO:scrapy.spidermiddlewares.httperror:Ignoring response <301 http://www.tradermade.com>: HTTP status code is not handled or not allowed
INFO:scrapy.spidermiddlewares.httperror:Ignoring response <301 http://www.bondcasebriefs.com>: HTTP status code is not handled or not allowed
DEBUG:my_spider:Page: http://www.northbabylonschools.net (
	North Babylon School District
)
INFO:scrapy.spidermiddlewares.httperror:Ign

DEBUG:my_spider:Page: http://www.examzone.com (Examzone | Pass Your Exam!)
DEBUG:my_spider:Page: http://www.thetacapital.com ( Theta Capital Management)
DEBUG:scrapy.core.engine:Crawled (200) <GET http://www.trueblueclean.com> (referer: None)
INFO:scrapy.spidermiddlewares.httperror:Ignoring response <302 http://www.elcbroward.org/p/3/about-the-coalition>: HTTP status code is not handled or not allowed
DEBUG:my_spider:Page: http://www.bestinc.org/b_about_best.php (Best Robotics Inc. - Boosting Engineering Science and Technology)
DEBUG:my_spider:Page: http://www.presentationxpert.com/about-us (About Us | Presentation Xpert)
DEBUG:my_spider:Page: http://www.northbabylonschools.net/faculty_resources/new_teacher_overview (
	North Babylon School District Faculty Resources | New Teacher Overview
)
DEBUG:scrapy.core.engine:Crawled (301) <GET http://www.vaircompanies.com> (referer: None)
DEBUG:scrapy.core.engine:Crawled (200) <GET http://www.aafcleveland.com/membership/overview/> (referer: http

INFO:scrapy.spidermiddlewares.httperror:Ignoring response <404 http://www.aafcleveland.com/about-us/leadership/tel:2169014000>: HTTP status code is not handled or not allowed
INFO:scrapy.spidermiddlewares.httperror:Ignoring response <404 http://www.aafcleveland.com/about-us/committees/tel:2169014000>: HTTP status code is not handled or not allowed
DEBUG:scrapy.core.engine:Crawled (404) <GET http://www.aafcleveland.com/about-us/leadership/tel:216901400> (referer: http://www.aafcleveland.com/about-us/leadership/)
INFO:scrapy.spidermiddlewares.httperror:Ignoring response <404 http://www.aafcleveland.com/about-us/leadership/tel:216901400>: HTTP status code is not handled or not allowed
DEBUG:scrapy.core.engine:Crawled (404) <GET http://www.aafcleveland.com/about-us/aaf-cleveland-advertising-hall-of-fame/tel:2169014000> (referer: http://www.aafcleveland.com/about-us/aaf-cleveland-advertising-hall-of-fame/)
DEBUG:scrapy.core.engine:Crawled (404) <GET http://www.aafcleveland.com/about-us/aaf-

INFO:scrapy.core.engine:Closing spider (finished)
INFO:scrapy.statscollectors:Dumping Scrapy stats:
{'downloader/exception_count': 11,
 'downloader/exception_type_count/twisted.internet.error.DNSLookupError': 6,
 'downloader/exception_type_count/twisted.internet.error.TimeoutError': 4,
 'downloader/exception_type_count/twisted.web._newclient.ResponseNeverReceived': 1,
 'downloader/request_bytes': 34445,
 'downloader/request_count': 136,
 'downloader/request_method_count/GET': 136,
 'downloader/response_bytes': 707113,
 'downloader/response_count': 125,
 'downloader/response_status_count/200': 54,
 'downloader/response_status_count/301': 50,
 'downloader/response_status_count/302': 8,
 'downloader/response_status_count/403': 1,
 'downloader/response_status_count/404': 12,
 'dupefilter/filtered': 67,
 'elapsed_time_seconds': 30.614818,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2019, 7, 30, 14, 53, 22, 290167),
 'httperror/response_ignored_count': 71,
 'httperror/res

CPU times: user 3.64 s, sys: 560 ms, total: 4.2 s
Wall time: 30.8 s


INFO:twisted:(TCP Port 6023 Closed)


In [1]:
len(data1)

Starting Spark application


The code failed because of a fatal error:
	Session 3 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
19/07/31 11:01:16 INFO RSCDriver: Connecting to: ip-10-0-3-246.eu-west-2.compute.internal:10000
19/07/31 11:01:16 INFO RSCDriver: Starting RPC server...
19/07/31 11:01:16 INFO RpcServer: Connected to the port 10001
19/07/31 11:01:16 WARN RSCConf: Your hostname, ip-10-0-3-246.eu-west-2.compute.internal, resolves to a loopback address, but we couldn't find any external IP address!
19/07/31 11:01:16 WARN RSCConf: Set livy.rsc.rpc.server.address if you need to bind to another address.
19/07/31 11:01:16 INFO RSCDriver: Received job request 05ef6139-f493-4f19-848d-9cc8df970b9e
19/07/31 11:01:16 INFO RSCDriver: SparkContext not yet up, queueing job request.
19/07/31 11:01:22 INFO SparkEntries: Starting Spark context...
19/07/31 11:01:22 INFO SparkContext: Running Spark version 2.4.2
19/07/31 11:01:22 INFO SparkContext: Submitted application: livy-session-3
19/07/31 11:01

In [10]:
for el in data1:
    if "http://www.thetacapital.com" in el[1]:
        print(el)

(' Theta Capital Management', 'http://www.thetacapital.com', ' Theta Capital Management', 'html', 'Home\n\nAbout Us\n\n  \n  \n\n# Theta Capital Management has been a leading provider of hedge fund solutions\nsince 2001.\n\n## Our team of specialists have over 100 years of combined hedge fund\nexperience which includes managing through the 2008 financial crisis. As\nspecialists in hedge fund investing we have a deep understanding of the space\nand focus on the areas where such strategies offer most added value and avoid\nthe pitfalls unclear to non-dedicated investors.\n\nTheta Capital Management and shareholders are the largest investors, fully\naligning ourselves with our investor base, which include family offices,\ninstitutions, endowments and select individuals. The Theta approach has\ntranslated into significant and consistent outperformance over time.\n\n##  A differentiating, value-added approach  \n\n#### Based on three pillars\n\n## Differentiated investments\n\nWe identify, 

In [7]:
crawl_df = spark.createDataFrame(df
            .rdd
            .mapPartitions(lambda u: run_spider(el.url for el in u))
            , T.StructType([
                T.StructField("website", T.StringType()),
                T.StructField("url", T.StringType()),
                T.StructField("page_title", T.StringType()),
                T.StructField("content_type", T.StringType()),
                T.StructField("raw_content", T.StringType())
            ])
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
crawl_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- website: string (nullable = true)
 |-- url: string (nullable = true)
 |-- page_title: string (nullable = true)
 |-- content_type: string (nullable = true)
 |-- raw_content: string (nullable = true)

In [9]:
(crawl_df
 .repartition(100)
 .write
 .parquet("s3://onai-ml-dev-eu-west-1/web_crawler/data/urls_and_content", mode="overwrite")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Error sending http request and maximum retry encountered.


In [15]:
tst = spark.read.load("s3://onai-ml-dev-eu-west-1/web_crawler/data/urls_and_content")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
tst.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------+--------------------+-----------------------------------+------------+-------------------------------+
|                            website|                 url|                         page_title|content_type|                    raw_content|
+-----------------------------------+--------------------+-----------------------------------+------------+-------------------------------+
|           东华工程科技股份有限公司|http://www.chinae...|                              about|        html|           |

|  |

Chinese ...|
|六合开奖记录-六合开奖结果-今晚六...|http://www.amerir...|     联系我们-六合开奖结果- Powe...|        html|      #

  * 设为主页
  * 加...|
|                           怡球集团|http://www.yechiu...|                                   |        html|   股票代码601388

简体中文 ...|
|                         RealEnergy|http://www.realen...|                 Board of Directors|        html|                             

|
|                             heerim|http://www.heerim...|                

In [17]:
tst.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8849

In [18]:
(tst
 .repartition(100)
 .write
 .parquet("s3://onai-ml-dev-eu-west-1/web_crawler/data/sample_urls_and_content", mode="overwrite")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Notes
* 10 urls per partition ~ 2 mins per partition: takes about 1.5 hrs to finish 10k urls
* 100 urls per partition and config fixes: 21 minutes
* 1000 urls per partition: 17 mins
* looking only at main page and about, takes 8 mins 100 docs per partition